<a href="https://colab.research.google.com/github/robins-machines/Deep-Learning-with-TensorFlow-and-Keras-3rd-edition/blob/main/Intro_to_Tensorflow_German_traffic_sign_recog_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'german-traffic-sign-recognition-benchmark:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1801850%2F2939054%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240601%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240601T090454Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0485006ca4918d126a85758dcafe563510364b3058d031262dedced96322ebbb58190c6f595e080230b3a2b6a92090d6ad20ee473fc7fc67e785adee51f5cfbaced9bba35b46393e6ac921a7c072873ddee07b891c7b4348332a2d84958ec5a8a27660da499e47598ac03c3daf3e9c2e1eacc7411bdfbbc6bdeaa9d8e11b3c064b1f68f7b626bae4235c465b4b8fbdf7789588447a5addaac0c2222a028f0c41b5334a25d0f750443d9957e09d750cdfb8dcf434b67de9779d7561344d4e356e111fb713122c8b92fc4c2b24a70b0d8094a51ada354ed19cd77b33bc3fabd9dfeb27926955ae4abec7d4677a7a0297fe9a0e38ddc505aa312a9987ee3b9c1208'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


This notebook was copied from, https://github.com/sniper0110/IntroductionToTensorflow2
Thank a lot to Nour for wonderful tutorial at
https://www.youtube.com/watch?v=cPmjQ9V6Hbk

In [ ]:
import shutil
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
import shutil
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, Dense, MaxPool2D, BatchNormalization, GlobalAvgPool2D, Flatten
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping



# Preprocessing

In [ ]:
def create_generators(batch_size, train_data_path, val_data_path, test_data_path):

    train_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
        rotation_range=10,
        width_shift_range=0.1
    )

    test_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
    )

    train_generator = train_preprocessor.flow_from_directory(
        train_data_path,
        class_mode="categorical",
        target_size=(60,60),
        color_mode='rgb',
        shuffle=True,
        batch_size=batch_size
    )

    val_generator = test_preprocessor.flow_from_directory(
        val_data_path,
        class_mode="categorical",
        target_size=(60,60),
        color_mode="rgb",
        shuffle=False,
        batch_size=batch_size,
    )

    test_generator = test_preprocessor.flow_from_directory(
        test_data_path,
        class_mode="categorical",
        target_size=(60,60),
        color_mode="rgb",
        shuffle=False,
        batch_size=batch_size,
    )

    return train_generator, val_generator, test_generator

# Model

In [ ]:
batch_size = 64

train_data_path = '../input/german-traffic-sign-recognition-benchmark/train'
val_data_path = '../input/german-traffic-sign-recognition-benchmark/val'
test_data_path = '../input/german-traffic-sign-recognition-benchmark/Test'

train_generator, val_generator, test_generator = create_generators(batch_size, train_data_path, val_data_path, test_data_path)

In [ ]:
nbr_classes = train_generator.num_classes
nbr_classes

In [ ]:
def streesigns_model(nbr_classes):

    my_input = Input(shape=(60,60,3))

    x = Conv2D(32, (3,3), activation='relu')(my_input)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, (3,3), activation='relu')(x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, (3,3), activation='relu')(x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)

    # x = Flatten()(x)
    x = GlobalAvgPool2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(nbr_classes, activation='softmax')(x)

    return Model(inputs=my_input, outputs=x)


In [ ]:
model = streesigns_model(nbr_classes=nbr_classes)
model.summary()

In [ ]:
# Make Models directory
if os.path.isdir('./Models'):
    os.makedirs('./Models')

In [ ]:
# Create Models directory
if not os.path.isdir('./Models'):
    os.makedirs('Models')

In [ ]:
os.listdir('./')

In [ ]:
epochs = 15
lr=0.0001

path_to_save_model = './Models/'
ckpt_saver = ModelCheckpoint(
    path_to_save_model,
    monitor="val_accuracy",
    mode='max',
    save_best_only=True,
    save_freq='epoch',
    verbose=1
)

early_stop = EarlyStopping(monitor="val_accuracy", patience=10)

model = streesigns_model(nbr_classes)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr, amsgrad=True)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Since we have trained model and saved it to avoid retraining I am making the switch called TRAIN
# If you wanna train make it True

TRAIN = False
if TRAIN:
    model.fit(train_generator,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=val_generator,
              callbacks=[ckpt_saver, early_stop]
              )


In [ ]:
history = model.history.history

# Plotting
plt.figure(figsize=(20,15))
plt.plot(history['loss'], label='Train Loss')
plt.plot(history['accuracy'], label='Train Accuracy')
plt.plot(history['val_loss'], label='Validation Loss')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.legend()

Looks like model might improve. Feel free to run it for more epochs or tweaking parameters :)

# Evaluate

In [ ]:
# Just to get to the directory where model belongs
os.chdir('/kaggle/working/')

model = tf.keras.models.load_model('./Models')
model.summary()

print("Evaluating validation set:")
model.evaluate(val_generator)

print("Evaluating test set : ")
model.evaluate(test_generator)

So on validation set 95% accuracy and on test set 85% accuracy. Was expecting better. Happy tweaking parameters :)

# Using model to predict the given image

In [ ]:
def predict_with_model(model, imgpath):

    image = tf.io.read_file(imgpath)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, [60,60]) # (60,60,3)
    image = tf.expand_dims(image, axis=0) # (1,60,60,3)

    predictions = model.predict(image) # [0.005, 0.00003, 0.99, 0.00 ....]
    predictions = np.argmax(predictions) # 2

    return predictions


In [ ]:
# Prediction
# Just to get to the directory where model belongs
os.chdir('/kaggle/working/')

model = tf.keras.models.load_model('./Models')

for i in range(20):
    label = np.random.randint(0,43)
    img_path = os.path.join('../input/german-traffic-sign-recognition-benchmark/Test/',str(label)+'/')
    image = np.random.choice(os.listdir(img_path))
    img_path = os.path.join(img_path, image)
    prediction = predict_with_model(model, img_path)

    print(f"Prediction = {prediction} and the label = {label}")
